In [1]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ultralytics import YOLO

In [2]:
TASK2METRIC = {
    "detect": "metrics/mAP50(B)", #detect 지표를 mAP50-95에서 50으로 변경하기 위함
    "segment": "metrics/mAP50-95(M)",
    "classify": "metrics/accuracy_top1",
    "pose": "metrics/mAP50-95(P)",
    "obb": "metrics/mAP50-95(B)",
}

In [3]:
experiment_name = 'tune'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# Load a YOLOv8n model
best_model_tuning = YOLO("yolov8n.pt")

task = best_model_tuning.task
metric = TASK2METRIC[task] 

result_grid = best_model_tuning.tune(
    data="/home/under1/Detect/jeongui/coronal/coronal_t1wce_2_class.yaml",
    epochs=200,
    use_ray=True,
    name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
    gpu_per_trial=1  # 각 트라이얼에 1개의 GPU 할당
)

(_tune pid=45046) New https://pypi.org/project/ultralytics/8.2.49 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=45045) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=45045) engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/under1/Detect/jeongui/coronal/coronal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=

  0%|          | 0.00/6.25M [00:00<?, ?B/s]
  6%|▌         | 352k/6.25M [00:00<00:01, 3.49MB/s]
 45%|████▍     | 2.80M/6.25M [00:00<00:00, 5.03MB/s]
 93%|█████████▎| 5.82M/6.25M [00:01<00:00, 4.10MB/s]
100%|██████████| 6.25M/6.25M [00:01<00:00, 4.23MB/s]
100%|██████████| 6.25M/6.25M [00:01<00:00, 3.70MB/s]
 95%|█████████▍| 5.91M/6.25M [00:02<00:00, 4.87MB/s]
100%|██████████| 6.25M/6.25M [00:02<00:00, 3.04MB/s]
train: Scanning /home/under1/Detect/dataset/coronal/labels/train.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
  0%|          | 0.00/6.25M [00:00<?, ?B/s] [repeated 3x across cluster]
100%|██████████| 6.25M/6.25M [00:02<00:00, 2.94MB/s] [repeated 59x across cluster]
 77%|███████▋  | 4.79M/6.25M [00:01<00:00, 3.99MB/s] [repeated 2x across cluster]


(_tune pid=45048) AMP: checks passed ✅
(_tune pid=45046) Overriding model.yaml nc=80 with nc=2 [repeated 3x across cluster]
(_tune pid=45046)                    from  n    params  module                                       arguments                      [repeated 3x across cluster]
(_tune pid=45046)  19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]               [repeated 21x across cluster]
(_tune pid=45046)   8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]            [repeated 12x across cluster]
(_tune pid=45046)   9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                  [repeated 3x across cluster]
(_tune pid=45046)  13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']           [repeated 6x across cluster]
(_tune pid=45046)  20             [-1, 9]  1         0  ultralytics.n

(_tune pid=45048) INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


(_tune pid=45048) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=45046) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=45047) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=45045) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

    200/200       2.1G   0.004136      1.768      1.312         11        640:  12%|█▎        | 2/16 [00:00<00:01, 10.91it/s]
    200/200       2.1G   0.004676      1.698      1.324         14        640:  25%|██▌       | 4/16 [00:00<00:01, 11.17it/s]
    200/200       2.1G   0.004709      1.662      1.285         13        640:  38%|███▊      | 6/16 [00:00<00:00, 10.83it/s]
    200/200       2.1G   0.004829       1.78       1.29         14        640:  50%|█████     | 8/16 [00:00<00:00, 10.76it/s]
    200/200       2.1G   0.004738      1.748      1.266         14        640:  62%|██████▎   | 10/16 [00:00<00:00, 11.35it/s]
    200/200       2.1G   0.004715       1.69      1.259         15        640:  75%|███████▌  | 12/16 [00:01<00:00, 11.27it/s]
    200/200       2.1G   0.004646      1.732      1.253         13        640:  88%|████████▊ | 14/16 [00:01<00:00, 11.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:

(_tune pid=1927)                    all         63         66       0.84      0.822      0.849      0.407
(_tune pid=1927) 
(_tune pid=1927) 200 epochs completed in 0.881 hours.
(_tune pid=1927) Optimizer stripped from runs/detect/tune/tune_raytune/weights/last.pt, 6.3MB
(_tune pid=1927) Optimizer stripped from runs/detect/tune/tune_raytune/weights/best.pt, 6.3MB
(_tune pid=1927) 
(_tune pid=1927) Validating runs/detect/tune/tune_raytune/weights/best.pt...
(_tune pid=1927) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=1927) Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:00<00:00,  6.05it/s]
    200/200       2.1G   0.004609      1.734      1.247         13        640: 100%|██████████| 16/16 [00:01<00:00, 10.99it/s] [repeated 9x across cluster]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.09it/s]


(_tune pid=1927)                    all         63         66      0.861      0.768      0.865      0.551
(_tune pid=1927)               negative         36         39      0.861      0.796      0.913      0.586
(_tune pid=1927)               positive         27         27      0.862      0.741      0.816      0.517
(_tune pid=1927) Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 1.8ms postprocess per image
(_tune pid=1927) Results saved to runs/detect/tune/tune_raytune


2024-07-05 05:10:03,486	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/under1/Detect/jeongui/coronal/runs/detect/tune2/_tune_2024-07-05_04-14-34' in 0.0171s.
2024-07-05 05:10:03,503	INFO tune.py:1041 -- Total run time: 3329.44 seconds (3329.38 seconds for the tuning loop).


In [4]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

Trial #0: Configuration: {'lr0': 0.05603004150512119, 'lrf': 0.8466753482840981, 'momentum': 0.6426912313032316, 'weight_decay': 0.0008002726752984986, 'warmup_epochs': 0.055182179255820474, 'warmup_momentum': 0.5979218748495558, 'box': 0.04326733047005575, 'cls': 2.8783225360875666, 'hsv_h': 0.03836495696332326, 'hsv_s': 0.7859552246914824, 'hsv_v': 0.4541520426108284, 'degrees': 38.90575880752869, 'translate': 0.4128304512640884, 'scale': 0.4989097138107523, 'shear': 3.6679114359579756, 'perspective': 0.0006427249277677906, 'flipud': 0.26703705328003224, 'fliplr': 0.3262635925888855, 'bgr': 0.6615622417785234, 'mosaic': 0.1928595065057187, 'mixup': 0.04929066324185705, 'copy_paste': 0.064297204110913, 'data': '/home/under1/Detect/jeongui/coronal/coronal_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}, Last Reported Metrics: {'metrics/precision(B)': 0.48157, 'metrics/recall(B)': 0.90557, 'metrics/mAP50(B)': 0.56809, 'metrics/mAP50-95(B)': 0.24595, 'val/box_loss': 0.01

In [5]:
# Extract the best trial configuration
best_result = result_grid.get_best_result(metric=metric, mode="max")
best_config = best_result.config
print("Best hyperparameters found were: ", best_config)

Best hyperparameters found were:  {'lr0': 0.058743909416433295, 'lrf': 0.7773592572735385, 'momentum': 0.7410888258445357, 'weight_decay': 0.0004947874908352815, 'warmup_epochs': 1.2045366217703224, 'warmup_momentum': 0.6623970318361742, 'box': 0.029701998766943016, 'cls': 1.056134618042269, 'hsv_h': 0.04421081964040399, 'hsv_s': 0.8759544362536916, 'hsv_v': 0.7036067041734066, 'degrees': 32.96459098094055, 'translate': 0.5421021656691869, 'scale': 0.4443048219133645, 'shear': 1.3986638165201049, 'perspective': 0.0002659411609162341, 'flipud': 0.053756908018382954, 'fliplr': 0.5338313292697838, 'bgr': 0.4002330857608879, 'mosaic': 0.988112866623811, 'mixup': 0.6129257095090133, 'copy_paste': 0.516141798810737, 'data': '/home/under1/Detect/jeongui/coronal/coronal_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}


In [6]:
experiment_name = 'train'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# 'data' 항목 제거
if 'data' in best_config:
    del best_config['data']
if 'name' in best_config:
    del best_config['name']

# 최고의 하이퍼파라미터로 모델 재훈련
best_model = YOLO("yolov8n.pt")  # 모델 재초기화
best_results = best_model.train(data="coronal_t1wce_2_class.yaml",
                                optimizer='Adam',
                                augment=True,
                                nms=True,
                                name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
                                **best_config)

New https://pypi.org/project/ultralytics/8.2.49 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coronal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train_raytune2, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

train: Scanning /home/under1/Detect/dataset/coronal/labels/train.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/under1/Detect/dataset/coronal/labels/val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/train_raytune2/labels.jpg... 
optimizer: Adam(lr=0.058743909416433295, momentum=0.7410888258445357) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0004947874908352815), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train/train_raytune2
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    200/200       2.1G     0.0065       3.84      1.554         13        640: 100%|██████████| 16/16 [00:01<00:00, 12.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.87it/s]

                   all         63         66      0.286      0.501      0.295     0.0923



200 epochs completed in 0.126 hours.
Optimizer stripped from runs/detect/train/train_raytune2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/train_raytune2/weights/best.pt, 6.3MB

Validating runs/detect/train/train_raytune2/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.52it/s]


                   all         63         66      0.415      0.715      0.441      0.186
              negative         36         39      0.513      0.615      0.505      0.209
              positive         27         27      0.318      0.815      0.378      0.162
Speed: 0.1ms preprocess, 10.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train/train_raytune2


In [7]:
# train
best_results.results_dict

{'metrics/precision(B)': 0.4154674768692921,
 'metrics/recall(B)': 0.7150997150997151,
 'metrics/mAP50(B)': 0.44121776922909595,
 'metrics/mAP50-95(B)': 0.18552990314764853,
 'fitness': 0.21109868975579327}

In [8]:
# val
experiment_name = 'val'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model = YOLO("/home/under1/Detect/jeongui/coronal/runs/detect/train/train_raytune3/weights/best.pt")
best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'))

FileNotFoundError: [Errno 2] No such file or directory: '/home/under1/Detect/jeongui/coronal/runs/detect/train/train_raytune3/weights/best.pt'

In [ ]:
# test
experiment_name = 'test'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="test")

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


val: Scanning /home/under1/Detect/dataset/coronal/labels/test.cache... 78 images, 0 backgrounds, 0 corrupt: 100%|██████████| 78/78 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.87it/s]


                   all         78         83       0.46      0.719       0.49      0.299
              negative         32         34      0.412      0.618      0.413      0.241
              positive         46         49      0.508      0.821      0.568      0.357
Speed: 0.2ms preprocess, 1.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/test/test_raytune3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f942833ace0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804